In [1]:
import numpy as np 
import pandas as pd

خواندن داده‌ها

In [3]:
data=pd.read_csv('water_dataX.csv',encoding="ISO-8859-1")
data.head()

,STATION CODE,LOCATIONS,STATE,Temp,D.O. (mg/l),PH,CONDUCTIVITY (µmhos/cm),B.O.D. (mg/l),NITRATENAN N+ NITRITENANN (mg/l),FECAL COLIFORM (MPN/100ml),TOTAL COLIFORM (MPN/100ml)Mean,year
0,1393,"DAMANGANGA AT D/S OF MADHUBAN, DAMAN",DAMAN & DIU,30.6,6.7,7.5,203,NAN,0.1,11,27,2014
1,1399,ZUARI AT D/S OF PT. WHERE KUMBARJRIA CANAL JOI...,GOA,29.8,5.7,7.2,189,2,0.2,4953,8391,2014
2,1475,ZUARI AT PANCHAWADI,GOA,29.5,6.3,6.9,179,1.7,0.1,3243,5330,2014
3,3181,RIVER ZUARI AT BORIM BRIDGE,GOA,29.7,5.8,6.9,64,3.8,0.5,5382,8443,2014
4,3182,RIVER ZUARI AT MARCAIM JETTY,GOA,29.5,5.8,7.3,83,1.9,0.4,3428,5500,2014


انتخاب ستون ها

In [9]:
df = data.drop(columns=["STATION CODE", "LOCATIONS", "STATE", "Temp", "year"]).copy()
df.sample(5)

,D.O. (mg/l),PH,CONDUCTIVITY (µmhos/cm),B.O.D. (mg/l),NITRATENAN N+ NITRITENANN (mg/l),FECAL COLIFORM (MPN/100ml),TOTAL COLIFORM (MPN/100ml)Mean
1081,6.3,7.5,10825,2.9,1.93,5,101
989,6.8,6.8,160,1.5,NAN,NAN,23
1794,7.8,6.69,255.9,2.754,0.348,13,140
967,5.6,7.5,1269,34.5,12.15,22000,60000
1867,6,7.55,214.5,3.125,0.895,2600,10950


جایگزینی مقادیر گمشده با میانگین

In [8]:
from sklearn.impute import SimpleImputer

In [11]:
# تبدیل مقادیر غیرعددی به NaN
df.replace(r'^\s*$', np.nan, regex=True, inplace=True)
# تبدیل همه ستون‌ها به مقدار عددی (اگر هنوز داده‌های غیرعددی وجود داشته باشد)
df = df.apply(pd.to_numeric, errors='coerce')

In [12]:
imputer = SimpleImputer(strategy='mean')
df.iloc[:, :] = imputer.fit_transform(df)
df.head()

,D.O. (mg/l),PH,CONDUCTIVITY (µmhos/cm),B.O.D. (mg/l),NITRATENAN N+ NITRITENANN (mg/l),FECAL COLIFORM (MPN/100ml),TOTAL COLIFORM (MPN/100ml)Mean
0,6.7,7.5,203.0,6.940049,0.1,11.0,27.0
1,5.7,7.2,189.0,2.000000,0.2,4953.0,8391.0
2,6.3,6.9,179.0,1.700000,0.1,3243.0,5330.0
3,5.8,6.9,64.0,3.800000,0.5,5382.0,8443.0
4,5.8,7.3,83.0,1.900000,0.4,3428.0,5500.0


نرمال‌سازی داده‌ها

In [13]:
from sklearn.preprocessing import MinMaxScaler

In [14]:
scaler = MinMaxScaler()
df.iloc[:, :] = scaler.fit_transform(df.iloc[:, :])
df.head()

,D.O. (mg/l),PH,CONDUCTIVITY (µmhos/cm),B.O.D. (mg/l),NITRATENAN N+ NITRITENANN (mg/l),FECAL COLIFORM (MPN/100ml),TOTAL COLIFORM (MPN/100ml)Mean
0,0.587719,0.000112,0.003084,0.012799,0.00092,4.036377e-08,5.282818e-08
1,0.500000,0.000107,0.002871,0.003555,0.00184,1.817471e-05,1.641782e-05
2,0.552632,0.000103,0.002718,0.002994,0.00092,1.189997e-05,1.042867e-05
3,0.508772,0.000103,0.000968,0.006924,0.00460,1.974889e-05,1.651957e-05
4,0.508772,0.000109,0.001257,0.003368,0.00368,1.257882e-05,1.076130e-05


تغییر نام ستون ها

In [20]:
df.columns = ['do','ph','co','bod','na','fc','tc']
df.sample()

,do,ph,co,bod,na,fc,tc
835,0.622807,0.000118,0.001638,0.005988,0.003864,2.385132e-07,2.680541e-07
